In [1]:
import json
import sqlite3

# Connect to (or create) the SQLite database
conn = sqlite3.connect('/home/rick/job-scraper/job_scraper/metabase/my_cleaned_data_for_metabase.db')
cursor = conn.cursor()

# Create the categories table
cursor.execute("""
CREATE TABLE IF NOT EXISTS categories (
    id INTEGER PRIMARY KEY,
    parentId INTEGER,
    titleFa TEXT,
    titleEn TEXT,
    urlParameter TEXT,
    "order" INTEGER,
    seoOrder INTEGER
);
""")

# Load the JSON dictionary from file
with open('/home/rick/job-scraper/job_scraper/metabase/jobcategories_map.json', encoding='utf-8') as f:
    data = json.load(f)

# Insert each category from the JSON into the table
for key, value in data.items():
    cursor.execute("""
        INSERT INTO categories (id, parentId, titleFa, titleEn, urlParameter, "order", seoOrder)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (
        int(key),
        value["parentId"],
        value["titleFa"],
        value["titleEn"],
        value["urlParameter"],
        value["order"],
        value["seoOrder"]
    ))

conn.commit()
conn.close()


In [1]:
import json
import sqlite3

# Connect to (or create) the SQLite database
conn = sqlite3.connect('/home/rick/job-scraper/job_scraper/metabase/my_cleaned_data_for_metabase.db')
cursor = conn.cursor()

# Create the categories table
cursor.execute("""
CREATE TABLE new_cleaned_jobs AS
WITH RECURSIVE split(job_id, cat_code, rest) AS (
  -- Split the comma-separated list from jobPostCategories_string
  SELECT
    cj.id,
    substr(cj.jobPostCategories_string || ',', 1, instr(cj.jobPostCategories_string || ',', ',') - 1) AS cat_code,
    substr(cj.jobPostCategories_string || ',', instr(cj.jobPostCategories_string || ',', ',') + 1) AS rest
  FROM cleaned_jobs cj
  UNION ALL
  SELECT
    job_id,
    substr(rest, 1, instr(rest, ',') - 1),
    substr(rest, instr(rest, ',') + 1)
  FROM split
  WHERE rest <> ''
),
computed AS (
  -- Compute the concatenated root category titles (where parentId is NULL)
  SELECT
    cj.id,
    group_concat(CASE WHEN c.parentId IS NULL THEN c.titleEn END, ', ') AS root_category_titles
  FROM cleaned_jobs cj
  LEFT JOIN split s ON cj.id = s.job_id
  LEFT JOIN categories c ON CAST(s.cat_code AS INTEGER) = c.id
  GROUP BY cj.id
)
SELECT
  cj.*,
  computed.root_category_titles
FROM cleaned_jobs cj
LEFT JOIN computed ON cj.id = computed.id;

""")


conn.commit()
conn.close()
